In [1]:
from gurobipy.gurobipy import Model, quicksum, GRB

In [2]:
import numpy as np

In [3]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [75]:
#Generate a set Y'
i = 0
Y = []
while i< 10:
    y = np.random.randint(2, size=28)
    if sum(y)< 16 and tuple(y) not in Y:
            Y.append(tuple(y))
            i = i+1

In [76]:
#Make I
I = list(powerset(Y))
I[1]

((0,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  0),)

In [77]:
#Make subset I'
I_prime = []
for y in Y:
    I_prime.append((y,))
I_prime[1]

((0,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  0),)

In [119]:
model = Model('EU')

In [120]:
Z = model.addVars(
    I_prime, name='Z', lb=0, ub=GRB.INFINITY
)

In [121]:
model.setObjective(
    quicksum(Z[i] for i in I_prime),
    sense=GRB.MINIMIZE
)

In [122]:
constraints = model.addConstrs(
    (quicksum(Z[i] for i in I_prime if y in i)>= 1 for y in Y),
    name = 'Constraint'
)

In [123]:
model.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 10 rows, 10 columns and 10 nonzeros
Model fingerprint: 0x7d02abb3
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 10 rows and 10 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.000000000e+01


In [124]:
if model.status != GRB.OPTIMAL:
    raise Exception(f'Optimizer exited with status {model.status}.')

In [125]:
print(f'Optimal objective value: {model.objVal}\n')
for i in Z.values():
    print(f'{i.varName} = {i.x}')

Optimal objective value: 10.0

Z[(0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0)] = 1.0
Z[(0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0)] = 1.0
Z[(1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0)] = 1.0
Z[(1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0)] = 1.0
Z[(1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1)] = 1.0
Z[(1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0)] = 1.0
Z[(1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0)] = 1.0
Z[(1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1)] = 1.0
Z[(1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1)] = 1.0
Z[(1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1)] = 1.0


In [126]:
alpha = []
for constraint in constraints.values () :
    alpha.append(constraint.pi)
alpha

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [100]:
#Select subset X' of X
i = 0
X = []
while i< 10:
    x = np.random.choice([0,1], size=28, p = [1./3, 2./3])
    if sum(x)>= 25 and tuple(x) not in X:
        X.append(tuple(x))
        i = i+1

In [160]:
model_1 = Model('EU_1')

In [161]:
index = [i for i in range(len(Y))]
vote = [i for i in range(28)]

In [166]:
U = model_1.addVars(index
    , name='U', vtype=GRB.BINARY 
)
a = model_1.addVars(vote, name='a', lb = -1, ub = 1)
beta = model_1.addVars([1], name='b', lb = -28, ub = 28)
epsilon = model_1.addVars([1], name='epsilon', lb = 0, ub = GRB.INFINITY)

In [167]:
model_1.setObjective(
    quicksum(alpha[i]*U[i] for i in index),
    sense=GRB.MAXIMIZE
)

In [169]:
constraint_1 = model_1.addConstrs(
    ((quicksum(a[i]*x[i] for i in vote)<= beta[1]) for x in X),
    name = 'Constraint_1'
)

In [180]:
constraint_2 = model_1.addConstr(
    ((U[i] == 1) >> (quicksum(a[k]*(Y[i][k]) for k in vote)>= beta[1]+epsilon[1]) for i in index),
    name = 'Constraint_2'
)

TypeError: unsupported operand type(s) for -: 'generator' and 'NoneType'